<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2025noteboks/2025_0919demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm

try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

np.random.seed(42)

In [ ]:
n_sample = 50
sigma = 0.5
x = np.linspace(0, 20, n_sample)
X3 = np.column_stack((x, np.sin(x), (x - 5) ** 2, np.ones(n_sample)))
X2 = np.column_stack((x, np.sin(x), (x - 5) ** 2))
X1 = np.column_stack((x, np.sin(x)))
X0 = np.column_stack((x, np.sin(x)))
theta3 = [0.5, 0.5, -0.02, 5.0]
theta2 = [0.5, 0.5, -0.02]
theta1 = [0.5, 0.5]
theta0 = [0.5]

y_true = np.dot(X3, theta3)
y = y_true + sigma * np.random.normal(size=n_sample)

In [ ]:
plt.plot(x,y)
plt.show()

In [ ]:
plt.plot(np.sin(x), c='red', label='X0')
#plt.plot((x-5) ** 2, c='blue', label='X1')
plt.plot(np.ones(n_sample), c='green', label='X2')
plt.legend()
plt.show()

In [ ]:
res3 = sm.OLS(y, X3).fit()
res2 = sm.OLS(y, X2).fit()
res1 = sm.OLS(y, X1).fit()
res0 = sm.OLS(y, X0).fit()

for _res in [res0, res1, res2, res3]:
    print(_res.summary())

In [ ]:
np.set_printoptions(precision=4, suppress=False)
for _res in [res0, res1, res2, res3]:
    print(f"Parameters: {_res.params}")
    print(f"Standard errors: {_res.bse}")
    print("-" * 77)
#print("Predicted values: ", res.predict())


In [ ]:
_res = res3
pred_ols = _res.get_prediction()
iv_l = pred_ols.summary_frame()["obs_ci_lower"]
iv_u = pred_ols.summary_frame()["obs_ci_upper"]

fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(x, y, "o", label="データ")
ax.plot(x, y_true, "b-", label="True")
ax.plot(x, _res.fittedvalues, "r--.", label="OLS")
ax.plot(x, iv_u, "r--")
ax.plot(x, iv_l, "r--")
ax.legend(loc="best")

In [ ]:
y_fft = np.fft.fft(y)
y_ifft = np.fft.ifft(y_fft)

plt.plot(x, y_ifft.real)
#plt.plot(x, y_fft.real)
plt.show()

In [ ]:
y

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length] # Sequence of length `seq_length`
        y = data[i+seq_length]   # Label is the next value
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 20 # Number of past time steps to look at

X, y = create_sequences(y, SEQ_LENGTH)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Convert arrays to PyTorch tensors
X_train = torch.Tensor(X_train).unsqueeze(-1) # Shape: (batch, seq, input_size)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test).unsqueeze(-1)
y_test = torch.Tensor(y_test)


In [ ]:
X_train.size(), y_train.size()

In [ ]:
class BasicRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1):
        super(BasicRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)         # RNN output for all time steps
        out = out[:, -1, :]          # Take output from the last time step
        return self.fc(out)          # Pass through linear layer

# Instantiate the model
model = BasicRNN()
model.eval()

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
EPOCHS = 100

losses = []
for epoch in range(EPOCHS):
    model.train()
    output = model(X_train)
    loss = criterion(output.squeeze(), y_train)  # Compute loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}")

In [ ]:
scaler = MinMaxScaler()
scaler = MinMaxScaler()
df['value'] = scaler.fit_transform(df[['value']])  # Normalize values


In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_test).squeeze().numpy() # Predict on test set

# Inverse transform to original scale
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions.reshape(-1, 1))

# Plot actual vs predicted
#plt.figure(figsize=(10, 4))
#plt.plot(y_test_inv, label='Actual')
#plt.plot(predictions_inv, label='Predicted')
#plt.title("RNN Prediction vs Actual")
#plt.xlabel("Time Step")
#plt.ylabel("Value")
#plt.legend()
#plt.show()

In [ ]:
#print(X_test, predictions)
#X_test.size()# , predictions.size()
print(y_test.reshape(-1,1).numpy(), predictions.reshape(-1,1))
plt.plot(y_test.reshape(-1,1).numpy(), predictions.reshape(-1,1))